In [0]:
!pip3 -qq install torch==0.4.1
!pip -qq install torchtext==0.3.1
!git clone https://github.com/MiuLab/SlotGated-SLU.git
!wget -qq https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/week08_multitask/conlleval.py

In [0]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


if torch.cuda.is_available():
    from torch.cuda import FloatTensor, LongTensor
    DEVICE = torch.device('cuda')
else:
    from torch import FloatTensor, LongTensor
    DEVICE = torch.device('cpu')

np.random.seed(42)

# Диалоговые системы

Диалоговые системы делятся на два типа - *goal-orientied* и *general conversation*.

**General conversation** - это болталка, разговор на свободную тему:  
![](https://i.ibb.co/bFwwGpc/alice.jpg =300x)

Сегодня будем говорить не про них, а про **goal-orientied** системы:
![](https://hsto.org/webt/gj/3y/xl/gj3yxlqbr7ujuqr9r2akacxmkee.jpeg =700x)  
*From [Как устроена Алиса](https://habr.com/company/yandex/blog/349372/)*

Пользователь говорит что-то, это что-то распознается. По распознанному определяется - что, где и когда он хотел. Дальше диалоговый движок решает, действительно ли пользователь знает, чего хотел попросить. Происходит поход в источники - узнать информацию, которую (кажется) запросил пользователь. Исходя из всего этого генерируется некоторый ответ:

![](https://i.ibb.co/8XcdpJ7/goal-orientied.png =900x)  
*From [Как устроена Алиса](https://habr.com/company/yandex/blog/349372/)*

([Клёвая гифка, на которой то же самое, но в динамике](https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/resources/task_oriented_dialog_systems.gif))

Будем учить ту часть, которая посередине - классификатор и теггер. Все остальное обычно - эвристики и захардкоженные ответы.

## Загрузка данных

Есть условно стандартный датасет - atis, который неприлично маленький, на самом деле.

К нему можно взять еще датасет snips - он больше и разнообразнее.

Оба датасета возьмем из репозитория статьи [Slot-Gated Modeling for Joint Slot Filling and Intent Prediction](http://aclweb.org/anthology/N18-2118).

Начнем с atis.

In [0]:
import os 

def read_dataset(path):
    with open(os.path.join(path, 'seq.in')) as f_words, \
            open(os.path.join(path, 'seq.out')) as f_tags, \
            open(os.path.join(path, 'label')) as f_intents:
        
        return [
            (words.strip().split(), tags.strip().split(), intent.strip()) 
            for words, tags, intent in zip(f_words, f_tags, f_intents)
        ]

In [0]:
train_data = read_dataset('SlotGated-SLU/data/atis/train/')
val_data = read_dataset('SlotGated-SLU/data/atis/valid/')
test_data = read_dataset('SlotGated-SLU/data/atis/test/')

In [0]:
intent_to_example = {example[2]: example for example in train_data}
for example in intent_to_example.values():
    print('Intent:\t', example[2])
    print('Text:\t', '\t'.join(example[0]))
    print('Tags:\t', '\t'.join(example[1]))
    print()

In [0]:
from torchtext.data import Field, LabelField, Example, Dataset, BucketIterator

tokens_field = Field()
tags_field = Field(unk_token=None)
intent_field = LabelField()

fields = [('tokens', tokens_field), ('tags', tags_field), ('intent', intent_field)]

train_dataset = Dataset([Example.fromlist(example, fields) for example in train_data], fields)
val_dataset = Dataset([Example.fromlist(example, fields) for example in val_data], fields)
test_dataset = Dataset([Example.fromlist(example, fields) for example in test_data], fields)

tokens_field.build_vocab(train_dataset)
tags_field.build_vocab(train_dataset)
intent_field.build_vocab(train_dataset)

print('Vocab size =', len(tokens_field.vocab))
print('Tags count =', len(tags_field.vocab))
print('Intents count =', len(intent_field.vocab))

train_iter, val_iter, test_iter = BucketIterator.splits(
    datasets=(train_dataset, val_dataset, test_dataset), batch_sizes=(32, 128, 128), 
    shuffle=True, device=DEVICE, sort=False
)

## Классификатор интентов

Начнем с классификатора: к какому интенту относится данный запрос.

**Задание** Ничего умного - возьмите rnn'ку и научитесь предсказывать метки-интенты.

In [0]:
class IntentClassifierModel(nn.Module):
    def __init__(self, vocab_size, intents_count, emb_dim=64, lstm_hidden_dim=128, num_layers=1):
        super().__init__()

        <init layers>

    def forward(self, inputs):
        <apply layers>

**Задание** `ModelTrainer` для подсчета лосса и accuracy.

In [0]:
class ModelTrainer():
    def __init__(self, model, criterion, optimizer):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        
    def on_epoch_begin(self, is_train, name, batches_count):
        """
        Initializes metrics
        """
        self.epoch_loss = 0
        self.correct_count, self.total_count = 0, 0
        self.is_train = is_train
        self.name = name
        self.batches_count = batches_count
        
        self.model.train(is_train)
        
    def on_epoch_end(self):
        """
        Outputs final metrics
        """
        return '{:>5s} Loss = {:.5f}, Accuracy = {:.2%}'.format(
            self.name, self.epoch_loss / self.batches_count, self.correct_count / self.total_count
        )
        
    def on_batch(self, batch):
        """
        Performs forward and (if is_train) backward pass with optimization, updates metrics
        """
        <As usual: perform the forward pass, then call backward and apply optimizer>

In [0]:
import math
from tqdm import tqdm
tqdm.get_lock().locks = []


def do_epoch(trainer, data_iter, is_train, name=None):
    trainer.on_epoch_begin(is_train, name, batches_count=len(data_iter))
    
    with torch.autograd.set_grad_enabled(is_train):
        with tqdm(total=trainer.batches_count) as progress_bar:
            for i, batch in enumerate(data_iter):
                batch_progress = trainer.on_batch(batch)

                progress_bar.update()
                progress_bar.set_description(batch_progress)
                
            epoch_progress = trainer.on_epoch_end()
            progress_bar.set_description(epoch_progress)
            progress_bar.refresh()

            
def fit(trainer, train_iter, epochs_count=1, val_iter=None):
    best_val_loss = None
    for epoch in range(epochs_count):
        name_prefix = '[{} / {}] '.format(epoch + 1, epochs_count)
        do_epoch(trainer, train_iter, is_train=True, name=name_prefix + 'Train:')
        
        if not val_iter is None:
            do_epoch(trainer, val_iter, is_train=False, name=name_prefix + '  Val:')

In [0]:
model = IntentClassifierModel(vocab_size=len(tokens_field.vocab), intents_count=len(intent_field.vocab)).to(DEVICE)

criterion = nn.CrossEntropyLoss().to(DEVICE)
optimizer = optim.Adam(model.parameters())

trainer = ModelTrainer(model, criterion, optimizer)

fit(trainer, train_iter, epochs_count=30, val_iter=val_iter)

**Задание** Подсчитайте итоговое качество на тесте.

## Теггер

![](https://commons.bmstu.wiki/images/0/00/NER1.png)  
*From [NER](https://ru.bmstu.wiki/NER_(Named-Entity_Recognition)*

**Задание** Всё ещё ничего умного - простой теггер, как POS, только NER.

In [0]:
class TokenTaggerModel(nn.Module):
    def __init__(self, vocab_size, tags_count, emb_dim=64, lstm_hidden_dim=128, num_layers=1):
        super().__init__()

        <init layers again>

    def forward(self, inputs):
        <apply 'em>

**Задание** Обновите `ModelTrainer`: считать нужно всё те же лосс и accuracy, только теперь немного по-другому.

In [0]:
<fit the model>

NER обычно оценивают по F1-скору угадывания слотов. Для этого все перетаскивают скрипт conlleval друг у друга :)

**Задание** Напишите функцию для оценки теггера.

In [0]:
from conlleval import evaluate

def eval_tagger(model, test_iter):
    true_seqs, pred_seqs = [], []

    model.eval()
    with torch.no_grad():
        for batch in test_iter:
            <calc true_seqs and pred_seqs for the batch>
    print('F1 = {:.2f}%, Precision = {:.2f}%, Recall = {:.2f}%'.format(*evaluate(true_seqs, pred_seqs, verbose=False)))

## Multi-task learning

Мы уже обсуждали - multi-task learning крут, моден и молодежен. Давайте ~~будем как он~~ реализуем модель, которая умеет сразу и предсказывать теги и интенты. Идея в том, что в этом всем есть общая информация, которая должна помочь как одной, так и другой задаче: зная интент, можно понять, какие слоты вообще могут быть, а зная слоты, можно угадать и интент.

**Задание** Реализуйте объединенную модель.

In [0]:
class SharedModel(nn.Module):
    def __init__(self, vocab_size, intents_count, tags_count, emb_dim=64, lstm_hidden_dim=128, num_layers=1):
        super().__init__()

        <init layers>

    def forward(self, inputs):
        <apply layers>

In [0]:
<update ModelTrainer>

In [0]:
<fit the model>

In [0]:
<calc intent accuracy>

In [0]:
<calc tags F1-score>

 ## Асинхронное обучение
 
 Вообще, всё затевалось именно из-за этого - асинхронное обучение multi-task модели.
 
Идея описана в статье [A Bi-model based RNN Semantic Frame Parsing Model for Intent Detection and Slot Filling](http://aclweb.org/anthology/N18-2050).

Начнем с такой модели:

![](https://i.ibb.co/N2T1X2f/2018-11-27-2-11-01.png =x400)

Основное отличие от того, что уже реализовали в том, в каком порядке все оптимизируется. Вместо объединенного обучения всех слоев, сети для теггера и для классификатора обучаются отдельно.

На каждом шаге обучения генерируются последовательности скрытых состояний $h^1$ и $h^2$ - для классификатора и для теггера.

Дальше сначала считаются потери от предсказания интента и делается шаг оптимизатора, а затем потери от предсказания теггов - и опять шаг оптимизатора.

**Задание** Реализуйте это.

In [0]:
class AsyncSharedModel(nn.Module):
    def __init__(self, vocab_size, intents_count, tags_count, emb_dim=64, lstm_hidden_dim=128, num_layers=1):
        super().__init__()

        <init layers>
        
    <do smth>

In [0]:
<update ModelTrainer somehow>

Нужно создать отдельные оптимизаторы для каждой части модели.

Отдельные параметры можно получить так:

In [0]:
model = AsyncSharedModel(
    vocab_size=len(tokens_field.vocab),
    intents_count=len(intent_field.vocab),
    tags_count=len(tags_field.vocab)
).to(DEVICE)

tags_parameters = [param for name, param in model.named_parameters() if not name.startswith('_intent')]
intent_parameters = [param for name, param in model.named_parameters() if not name.startswith('_tags')]

Затем их нужно передать в отдельные оптимизаторы и учить отдельно.

*Еще, может быть, пригодится retain_graph параметр метода backward()*.

In [0]:
<fit the model>

In [0]:
<calc intent accuracy and tags F1-score>

## Улучшения

**Задание** Посмотрите на параметры в статье и попробуйте добиться похожего качества.

**Задание** Попробуйте заменить корпус, с которым работаете.

### Encoder-decoder

Хорошая идея - использовать не просто независимые предсказания тегов, а декодер над ними:

![](https://i.ibb.co/qrgVSqF/2018-11-27-2-11-17.png =600x)

По сути тут добавляется просто еще слой RNN - на этот раз однонаправленной. При этом его вход в случае предсказания тегов - это предыдущий тег, предыдущее скрытое состояние и скрытые состояния из энкодеров теггов и интента. Для интента - простая RNN.

**Задание** Реализуйте такую модель.

# Async Multi-task Learning for POS Tagging

Это были игрушечные датасеты и не самые хорошие статьи (хоть и с NAACL-2018).

Мне больше нравится вот эта: [Morphosyntactic Tagging with a Meta-BiLSTM Model over Context Sensitive Token Encodings](https://arxiv.org/pdf/1805.08237.pdf). Гораздо больше.

Архитектура там такая:

![](https://i.ibb.co/0nSX6CC/2018-11-27-9-26-15.png =x400)

Multi-task задача - обучение отдельных классификаторов более низкого уровня (над символами и словами) для предсказания тегов отдельными оптимизаторами.

**Задание** Попробовать реализовать, о чем в статье пишется.

# Дополнительные материалы

## Статьи
A Bi-model based RNN Semantic Frame Parsing Model for Intent Detection and Slot Filling, 2018 [[pdf]](http://aclweb.org/anthology/N18-2050)  
Slot-Gated Modeling for Joint Slot Filling and Intent Prediction, 2018 [[pdf]](http://aclweb.org/anthology/N18-2118)  
Morphosyntactic Tagging with a Meta-BiLSTM Model over Context Sensitive Token Encodings, 2018 [[arxiv]](https://arxiv.org/pdf/1805.08237.pdf)

## Блоги
[Как устроена Алиса](https://habr.com/company/yandex/blog/349372/)  

# Сдача

[Форма для сдачи](https://goo.gl/forms/4W7JDuSg3A32Ple72)  
[Feedback](https://goo.gl/forms/9aizSzOUrx7EvGlG3)